<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllCF_Amp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('all')
device_name = tf.test.gpu_device_name()
print(device_name)
df = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/InfluenceAnalysis/crowdFlower/crowdFlower_clean.csv')
df = df.sample(frac=1,random_state=32)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading pack

In [3]:
category_dict = {0:[1,0,0,0,0,0,0,0,0,0,0,0,0],1:[0,1,0,0,0,0,0,0,0,0,0,0,0],2:[0,0,1,0,0,0,0,0,0,0,0,0,0],3:[0,0,0,1,0,0,0,0,0,0,0,0,0],4:[0,0,0,0,1,0,0,0,0,0,0,0,0],5:[0,0,0,0,0,1,0,0,0,0,0,0,0],6:[0,0,0,0,0,0,1,0,0,0,0,0,0],7:[0,0,0,0,0,0,0,1,0,0,0,0,0],8:[0,0,0,0,0,0,0,0,1,0,0,0,0],9:[0,0,0,0,0,0,0,0,0,1,0,0,0],10:[0,0,0,0,0,0,0,0,0,0,1,0,0],11:[0,0,0,0,0,0,0,0,0,0,0,1,0],12:[0,0,0,0,0,0,0,0,0,0,0,0,1]}
data_tweet = [x.lower().split() for x in df['tweets']]
data_cat = np.array([category_dict[x] for x in df['emotion']])


In [4]:
print(data_tweet[:5])
print(data_cat[:5])

[['katofawesome', 'praying', 'for', 'love', 'in', 'a', 'lap', 'dance', 'and', 'paying', 'in', 'naivety', 'lt--one', 'of', 'my', 'fav', 'songs'], ['trying', 'to', 'fix', 'my', 'internet', 'connectionguess', 'my', 'prayers', 'have', 'been', 'answered', 'and', 'i', 'wont', 'have', 'any', 'study', 'distractions', 'ugh'], ['starting', 'an', 'account', 'here', 'on', 'twitter'], ['dipfico', 'hmm', 'wrong', 'link', 'ignore', 'my', 'tweet'], ['mzunyque', 'thanks', 'before', 'the', 'major', 'chop']]
[[0 0 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0]]


In [5]:
# Parameters
Min_count = 0
Embedding_size = 200
Window_size = 5
Negative_sampling = 00

In [6]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)
w2v_sg.wv.init_sims(True)
w2v_cbow.wv.init_sims(True)

In [7]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [8]:
batch_size = 100
Max_input_size = max([len(x) for x in data_tweet])
data_tweet = [nltk.pos_tag(x) for x in data_tweet]

In [9]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(w2v.wv.word_vec(i[0]),i[1]) for i in x if i[0] in vocab]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)          
      
def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor=2):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      res=[]
      for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]:
        tweet = [(vocab[i[0]],i[1]) for i in x if i[0] in vocab.keys()]
        l=len(tweet)
        for i in range(l):
          if tweet[i][1][:2]=='JJ' or tweet[i][1][:2]=='RB':
            tweet[i]=((tweet[i][0])*mul_factor,tweet[i][1])
        res+=[np.array([x[0] for x in tweet])]
      temp = np.array([np.pad(z.flatten(),(0,Max_input_size*Embedding_size-len(z.flatten()))).reshape(Max_input_size,Embedding_size) for z in res])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(data_tweet, data_cat, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=42)
result_table = [0,0,0,0,0,0,0,0,0]

In [11]:
epochs = 40

In [12]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
sg_train_gen=vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size)
sg_val_gen=vec_gen(w2v_sg,vocab_sg,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_sg_lstm = model_sg.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[0]=history[1]
print("SG_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 67s 254ms/step - loss: 2.2616 - accuracy: 0.2081 - val_loss: 2.1251 - val_accuracy: 0.2702
Epoch 2/40
257/257 [==============================] - 63s 247ms/step - loss: 2.1220 - accuracy: 0.2565 - val_loss: 2.0738 - val_accuracy: 0.2651
Epoch 3/40
257/257 [==============================] - 64s 250ms/step - loss: 2.0763 - accuracy: 0.2655 - val_loss: 2.0544 - val_accuracy: 0.2780
Epoch 4/40
257/257 [==============================] - 64s 251ms/step - loss: 2.0524 - accuracy: 0.2784 - val_loss: 2.0442 - val_accuracy: 0.2855
Epoch 5/40
257/257 [==============================] - 65s 254ms/step - loss: 2.0342 - accuracy: 0.2928 - val_loss: 2.0349 - val_accuracy: 0.2903
Epoch 6/40
257/257 [==============================] - 64s 249ms/step - loss: 2.0205 - accuracy: 0.2994 - val_loss: 2.0195 - val_accuracy: 0.3018
Epoch 7/40
257/257 [==============================] - 66s 256ms/step - loss: 2.0012 - accuracy: 0.3109 - val_loss: 2.0111 - val_ac

In [13]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_bi = model_sg_bi.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[1]=history[1]
print("SG_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 78s 292ms/step - loss: 2.2400 - accuracy: 0.2269 - val_loss: 2.0609 - val_accuracy: 0.2788
Epoch 2/40
257/257 [==============================] - 74s 289ms/step - loss: 2.0581 - accuracy: 0.2855 - val_loss: 2.0290 - val_accuracy: 0.2992
Epoch 3/40
257/257 [==============================] - 74s 290ms/step - loss: 2.0202 - accuracy: 0.3027 - val_loss: 2.0133 - val_accuracy: 0.3105
Epoch 4/40
257/257 [==============================] - 78s 303ms/step - loss: 1.9978 - accuracy: 0.3153 - val_loss: 2.0014 - val_accuracy: 0.3149
Epoch 5/40
257/257 [==============================] - 79s 309ms/step - loss: 1.9762 - accuracy: 0.3251 - val_loss: 1.9939 - val_accuracy: 0.3168
Epoch 6/40
257/257 [==============================] - 78s 302ms/step - loss: 1.9562 - accuracy: 0.3302 - val_loss: 1.9880 - val_accuracy: 0.3168
Epoch 7/40
257/257 [==============================] - 80s 314ms/step - loss: 1.9369 - accuracy: 0.3367 - val_loss: 1.9857 - val_ac

In [14]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_sg_rnn = model_sg_rnn.fit_generator(sg_train_gen,validation_data=sg_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)


result_table[2]=history[1]
print("SG_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 64s 247ms/step - loss: 2.2526 - accuracy: 0.1851 - val_loss: 2.1396 - val_accuracy: 0.2157
Epoch 2/40
257/257 [==============================] - 62s 241ms/step - loss: 2.1423 - accuracy: 0.2149 - val_loss: 2.1367 - val_accuracy: 0.2154
Epoch 3/40
257/257 [==============================] - 61s 238ms/step - loss: 2.1433 - accuracy: 0.2167 - val_loss: 2.1379 - val_accuracy: 0.2158
Epoch 4/40
257/257 [==============================] - 61s 239ms/step - loss: 2.1440 - accuracy: 0.2165 - val_loss: 2.1396 - val_accuracy: 0.2152
Epoch 5/40
257/257 [==============================] - 63s 245ms/step - loss: 2.1408 - accuracy: 0.2157 - val_loss: 2.1383 - val_accuracy: 0.2157
Epoch 6/40
257/257 [==============================] - 62s 242ms/step - loss: 2.1405 - accuracy: 0.2165 - val_loss: 2.1380 - val_accuracy: 0.2143
Epoch 7/40
257/257 [==============================] - 63s 247ms/step - loss: 2.1374 - accuracy: 0.2211 - val_loss: 2.1384 - val_ac

In [15]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
cbow_train_gen=vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size)
cbow_val_gen=vec_gen(w2v_cbow,vocab_cbow,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_cbow_lstm = model_cbow.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[3]=history[1]
print("CBOW_LSTM")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 66s 252ms/step - loss: 2.2525 - accuracy: 0.2127 - val_loss: 2.1392 - val_accuracy: 0.2149
Epoch 2/40
257/257 [==============================] - 64s 249ms/step - loss: 2.1467 - accuracy: 0.2147 - val_loss: 2.1363 - val_accuracy: 0.2158
Epoch 3/40
257/257 [==============================] - 65s 252ms/step - loss: 2.1443 - accuracy: 0.2215 - val_loss: 2.1002 - val_accuracy: 0.2588
Epoch 4/40
257/257 [==============================] - 64s 251ms/step - loss: 2.0978 - accuracy: 0.2539 - val_loss: 2.0572 - val_accuracy: 0.2826
Epoch 5/40
257/257 [==============================] - 64s 248ms/step - loss: 2.0547 - accuracy: 0.2763 - val_loss: 2.0430 - val_accuracy: 0.2912
Epoch 6/40
257/257 [==============================] - 63s 246ms/step - loss: 2.0337 - accuracy: 0.2913 - val_loss: 2.0282 - val_accuracy: 0.3034
Epoch 7/40
257/257 [==============================] - 64s 250ms/step - loss: 2.0115 - accuracy: 0.3063 - val_loss: 2.0222 - val_ac

In [16]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_bi = model_cbow_bi.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[4]=history[1]
print("CBOW_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 77s 289ms/step - loss: 2.2381 - accuracy: 0.2219 - val_loss: 2.0626 - val_accuracy: 0.2782
Epoch 2/40
257/257 [==============================] - 73s 285ms/step - loss: 2.0592 - accuracy: 0.2833 - val_loss: 2.0271 - val_accuracy: 0.3028
Epoch 3/40
257/257 [==============================] - 74s 288ms/step - loss: 2.0186 - accuracy: 0.3017 - val_loss: 2.0124 - val_accuracy: 0.3100
Epoch 4/40
257/257 [==============================] - 74s 287ms/step - loss: 1.9971 - accuracy: 0.3126 - val_loss: 2.0050 - val_accuracy: 0.3111
Epoch 5/40
257/257 [==============================] - 75s 291ms/step - loss: 1.9749 - accuracy: 0.3232 - val_loss: 2.0009 - val_accuracy: 0.3111
Epoch 6/40
257/257 [==============================] - 76s 295ms/step - loss: 1.9538 - accuracy: 0.3292 - val_loss: 1.9971 - val_accuracy: 0.3145
Epoch 7/40
257/257 [==============================] - 76s 296ms/step - loss: 1.9345 - accuracy: 0.3350 - val_loss: 1.9946 - val_ac

In [17]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_cbow_rnn = model_cbow_rnn.fit_generator(cbow_train_gen,validation_data=cbow_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[5]=history[1]
print("CBOW_RNN")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 58s 225ms/step - loss: 2.2220 - accuracy: 0.2075 - val_loss: 2.1389 - val_accuracy: 0.2154
Epoch 2/40
257/257 [==============================] - 57s 224ms/step - loss: 2.1422 - accuracy: 0.2147 - val_loss: 2.1344 - val_accuracy: 0.2154
Epoch 3/40
257/257 [==============================] - 58s 226ms/step - loss: 2.1422 - accuracy: 0.2167 - val_loss: 2.1358 - val_accuracy: 0.2148
Epoch 4/40
257/257 [==============================] - 57s 224ms/step - loss: 2.1402 - accuracy: 0.2210 - val_loss: 2.1454 - val_accuracy: 0.2258
Epoch 5/40
257/257 [==============================] - 57s 224ms/step - loss: 2.1319 - accuracy: 0.2287 - val_loss: 2.1481 - val_accuracy: 0.2218
Epoch 6/40
257/257 [==============================] - 58s 225ms/step - loss: 2.1277 - accuracy: 0.2365 - val_loss: 2.1549 - val_accuracy: 0.2218
Epoch 7/40
257/257 [==============================] - 58s 226ms/step - loss: 2.1235 - accuracy: 0.2430 - val_loss: 2.1547 - val_ac

In [18]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
glove_train_gen=glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size)
glove_val_gen=glove_gen(vocab_glove,X_val,y_val,batch_size,Max_input_size,Embedding_size)
history_glove_lstm = model_glove.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[6]=history[1]
print("GLOVE_LSTM")


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 15s 54ms/step - loss: 2.2439 - accuracy: 0.2289 - val_loss: 1.9783 - val_accuracy: 0.3105
Epoch 2/40
257/257 [==============================] - 13s 52ms/step - loss: 1.9598 - accuracy: 0.3207 - val_loss: 1.9030 - val_accuracy: 0.3523
Epoch 3/40
257/257 [==============================] - 13s 52ms/step - loss: 1.8909 - accuracy: 0.3536 - val_loss: 1.8785 - val_accuracy: 0.3578
Epoch 4/40
257/257 [==============================] - 13s 52ms/step - loss: 1.8522 - accuracy: 0.3720 - val_loss: 1.8705 - val_accuracy: 0.3578
Epoch 5/40
257/257 [==============================] - 13s 52ms/step - loss: 1.8247 - accuracy: 0.3789 - val_loss: 1.8666 - val_accuracy: 0.3608
Epoch 6/40
257/257 [==============================] - 13s 52ms/step - loss: 1.8012 - accuracy: 0.3901 - val_loss: 1.8597 - val_accuracy: 0.3628
Epoch 7/40
257/257 [==============================] - 13s 52ms/step - loss: 1.7701 - accuracy: 0.4017 - val_loss: 1.8677 - val_accuracy:

In [19]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_bi = model_glove_bi.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[7]=history[1]
print("GLOVE_BI")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/40
257/257 [==============================] - 27s 92ms/step - loss: 2.1361 - accuracy: 0.2640 - val_loss: 1.8922 - val_accuracy: 0.3445
Epoch 2/40
257/257 [==============================] - 23s 90ms/step - loss: 1.8820 - accuracy: 0.3490 - val_loss: 1.8474 - val_accuracy: 0.3634
Epoch 3/40
257/257 [==============================] - 23s 90ms/step - loss: 1.8198 - accuracy: 0.3759 - val_loss: 1.8385 - val_accuracy: 0.3675
Epoch 4/40
257/257 [==============================] - 23s 90ms/step - loss: 1.7823 - accuracy: 0.3878 - val_loss: 1.8358 - val_accuracy: 0.3698
Epoch 5/40
257/257 [==============================] - 23s 90ms/step - loss: 1.7491 - accuracy: 0.4018 - val_loss: 1.8410 - val_accuracy: 0.3702
Epoch 6/40
257/257 [==============================] - 23s 90ms/step - loss: 1.7126 - accuracy: 0.4138 - val_loss: 1.8604 - val_accuracy: 0.3688
Epoch 7/40
257/257 [==============================] - 23s 90ms/step - loss: 1.6769 - accuracy: 0.4300 - val_loss: 1.8769 - val_accuracy:

In [20]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(13, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history_glove_rnn = model_glove_rnn.fit_generator(glove_train_gen,validation_data=glove_val_gen,validation_steps=int((len(X_val)/batch_size)+1),steps_per_epoch=int((len(X_train)/batch_size)+1),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size),steps=int(len(y_test)/batch_size))
print(history)

result_table[8]=history[1]
print("GLOVE_RNN")

Epoch 1/40


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


257/257 [==============================] - 7s 25ms/step - loss: 2.2905 - accuracy: 0.1942 - val_loss: 2.1396 - val_accuracy: 0.2155
Epoch 2/40
257/257 [==============================] - 6s 24ms/step - loss: 2.1426 - accuracy: 0.2161 - val_loss: 2.1365 - val_accuracy: 0.2154
Epoch 3/40
257/257 [==============================] - 6s 24ms/step - loss: 2.1423 - accuracy: 0.2168 - val_loss: 2.1373 - val_accuracy: 0.2158
Epoch 4/40
257/257 [==============================] - 6s 24ms/step - loss: 2.1395 - accuracy: 0.2250 - val_loss: 2.1392 - val_accuracy: 0.2100
Epoch 5/40
257/257 [==============================] - 6s 24ms/step - loss: 2.1178 - accuracy: 0.2424 - val_loss: 2.0590 - val_accuracy: 0.2605
Epoch 6/40
257/257 [==============================] - 6s 24ms/step - loss: 2.0443 - accuracy: 0.2639 - val_loss: 2.0383 - val_accuracy: 0.2694
Epoch 7/40
257/257 [==============================] - 6s 24ms/step - loss: 2.0096 - accuracy: 0.2901 - val_loss: 2.0216 - val_accuracy: 0.2803
Epoch 8/40

In [21]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])
# print(history_sg_bi.history)
# plt.title("Skip-Gram + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_sg_bi.history['loss'],c='b',label='loss')
# plt.plot(history_sg_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()
# plt.title("GloVe + Bi-LSTM accuracy, loss vs epochs Graph")
# plt.plot(history_glove_bi.history['loss'],c='b',label='loss')
# plt.plot(history_glove_bi.history['accuracy'],c='r',label='accuracy')
# plt.xlabel('epochs')
# plt.legend()
# # plt.yticks()
# plt.show()

	Lstm			BiLstm			Rnn
sg [0.31349998712539673, 0.27562499046325684, 0.21799999475479126]
cbow [0.3023749887943268, 0.2706249952316284, 0.20212499797344208]
glove [0.31562501192092896, 0.2787500023841858, 0.21799999475479126]
